In [1]:
import pandas as pd
import numpy as np

import re

from utils.wordle import Wordle

# playing wordle by statistics

In [2]:
encoding_table = {
    '2': 'O',
    '1': '-',
    '0': 'X'
}


def encode_score(score, encoding_table):
    score_str = "".join([str(i) for i in score])
    for key in list(encoding_table.keys()):
        score_str = re.sub(key, encoding_table[key], score_str)
    return score_str


def decode_score(score_str, encoding_table):
    score = []
    decode_table = dict()
    for key in encode_table.keys():
        decode_table[encode_table[key]] = key
    for s in score_str:
        score.append(int(decode_table[s]))
    return score

In [3]:
# Wordle Game Setting

wordle_file = "./data/common 5-letter words.csv"

wd = Wordle(wordle_file)
wd.ans

'awoke'

In [4]:
df = pd.read_csv(wordle_file)
df.head()

,word,count
0,aaron,7
1,aback,2
2,abbas,2
3,abbey,7
4,abbot,2


# Create Score Matrix

In [5]:
vocabs = list(df['word'])

score_matrix = []
for ans in df['word']:\

    # using function from wordle core
    wd = Wordle(wordle_file)

    # score for this ans
    score_arr = []
    for guess in df['word']:
        encoded_score = encode_score(wd.get_score(ans, guess), encoding_table)
        score_arr.append(encoded_score)
    
    score_matrix.append(score_arr)

In [6]:
df = pd.DataFrame(columns=[vocabs], data=score_matrix)

In [7]:
df['possible_word'] = vocabs

In [8]:
def filter_possible_answer(df, possible_index, guess, guess_result):
    # filter answer, new possible index
    # based on current possible index and guess
    
    # look at the guess word column, and the possible answer
    pos_df = df[df.loc[possible_index, guess] == guess_result][guess].dropna()
    
    possible_words = df['possible_word'].loc[pos_df.index]
    return list(pos_df.index), np.array(possible_words).squeeze().tolist()


# Play

In [33]:
wd = Wordle(wordle_file)
print("ANS:", wd.ans)


# first guess
guess = np.random.choice(vocabs, 1)[0]
wd_result = wd.guess(guess)
guess_result = encode_score(wd_result['score'], encoding_table)

possible_ind = range(len(df))

while not (wd_result['score'] == 2).all():
    print(f"""guess: {wd_result['#guess']} | {wd_result['guessed word']} | {list(wd_result['score'])}""")
    
    possible_ind, possible_words = filter_possible_answer(df, possible_ind, guess, guess_result)
    
    # TODO: return as 1d list
    if type(possible_words) != str:
        guess = np.random.choice(possible_words, 1)[0]
        wd_result = wd.guess(guess)
    else:
        wd_result = wd.guess(possible_words)
    guess_result = encode_score(wd_result['score'], encoding_table)
    
print(f"""guess: {wd_result['#guess']} | {wd_result['guessed word']} | {list(wd_result['score'])}""")

ANS: drone
guess: 1 | messy | [0, 1, 0, 0, 0]
guess: 2 | dixie | [2, 0, 0, 0, 2]
guess: 3 | duque | [2, 0, 0, 0, 2]
guess: 4 | dance | [2, 0, 1, 0, 2]
guess: 5 | drone | [2, 2, 2, 2, 2]
